In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch.cuda.amp as amp
import os
import numpy as np
import itertools
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
bases_path = "/content/drive/MyDrive/HackIA_2024_GRP_2/Dataset/"
models_path = "/content/drive/MyDrive/HackIA_2024_GRP_2/Models/"
results_path = "/content/drive/MyDrive/HackIA_2024_GRP_2/Results/"

train_dataset_path = bases_path + "personal_Cleansed"
test_dataset_path = bases_path + "test/"

In [ ]:
# Déclarations des modèles et de leurs poids
model_loaders = {
    "VGG16_Weights": {
        "model": models.vgg16,
        "weights": models.VGG16_Weights.IMAGENET1K_V1
    },
    "ResNet18_Weights": {
        "model": models.resnet18,
        "weights": models.ResNet18_Weights.IMAGENET1K_V1
    },
    "MobileNet_V3_Small_Weights": {
        "model": models.mobilenet_v3_small,
        "weights": models.MobileNet_V3_Small_Weights.IMAGENET1K_V1
    },
    "MobileNet_V2_Weights": {
        "model": models.mobilenet_v2,
        "weights": models.MobileNet_V2_Weights.IMAGENET1K_V1
    },
    "MobileNet_V3_Large_Weights": {
        "model": models.mobilenet_v3_large,
        "weights": models.MobileNet_V3_Large_Weights.IMAGENET1K_V1
    },
    "EfficientNet_V2_S_Weights": {
        "model": models.efficientnet_v2_s,
        "weights": models.EfficientNet_V2_S_Weights.IMAGENET1K_V1
    }
}

In [ ]:
# Transformations pour les données d'entraînement et de validation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Charger les données d'entraînement et de test
train_dataset = datasets.ImageFolder(root=train_dataset_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dataset_path, transform=transform)

In [ ]:
# Créer des indices pour le split train/validation
validation_split = 0.2
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
train_indices, val_indices = train_test_split(indices, test_size=validation_split, random_state=42)

In [ ]:
# Création des samplers
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

In [ ]:
# DataLoader pour le test
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=4)

In [ ]:
# Définir les hyperparamètres à explorer
learning_rates = [0.001, 0.01]
hidden_units_list = [256, 512, 1024]
momentums = [0.95]
batch_sizes = [16, 32, 64]
weight_decays = [0.0001, 0.001]
classifiers = list(model_loaders.keys())  # Utiliser les clés du dictionnaire model_loaders

In [ ]:
# Déplacer le modèle sur le GPU si disponible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(classifier, lr, hidden_units, momentum, batch_size, weight_decay):
    print(f'Training with classifier={classifier}, learning_rate={lr}, hidden_units={hidden_units}, momentum={momentum}, batch_size={batch_size}, weight_decay={weight_decay}')

    # Charger le modèle
    model_info = model_loaders[classifier]
    model = model_info["model"](weights=model_info["weights"])

    # Adapter les classificateurs en fonction de la structure de chaque modèle
    if classifier in ["MobileNet_V2_Weights"]:
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 3)
    elif classifier in ["MobileNet_V3_Small_Weights", "MobileNet_V3_Large_Weights"]:
        model.classifier[3] = nn.Linear(model.classifier[3].in_features, 3)
    elif classifier in ["VGG16_Weights"]:
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 3)
    elif classifier in ["ResNet18_Weights"]:
        model.fc = nn.Linear(model.fc.in_features, 3)
    elif classifier in ["EfficientNet_V2_S_Weights"]:
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 3)

    model = model.to(device)

    # Geler les premières couches du modèle
    for param in model.parameters():
        param.requires_grad = False

    # Dégeler les couches classifier ou fc
    if hasattr(model, 'classifier'):
        for param in model.classifier.parameters():
            param.requires_grad = True
    elif hasattr(model, 'fc'):
        for param in model.fc.parameters():
            param.requires_grad = True

    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()
    scaler = amp.GradScaler()

    # Création des DataLoaders avec la taille de batch spécifiée
    trainloader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=4)
    valloader = DataLoader(train_dataset, batch_size=batch_size, sampler=val_sampler, num_workers=4)

    best_val_accuracy = 0.0
    best_model_state = None

    # Entraîner le modèle
    for epoch in range(5):  # Nombre fixe d'époques pour simplifier
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for data in tqdm(trainloader, desc=f"Training Epoch {epoch+1}"):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            with amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(trainloader)
        train_accuracy = correct / total
        print(f'Epoch {epoch + 1} Train Loss: {train_loss:.3f} Accuracy: {train_accuracy:.3f}')

        # Évaluer le modèle
        model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for data in tqdm(valloader, desc="Validation"):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                with amp.autocast():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss = running_loss / len(valloader)
        val_accuracy = correct / total
        print(f'Validation Loss: {val_loss:.3f} Accuracy: {val_accuracy:.3f}')

        # Sauvegarder le meilleur modèle basé sur la validation
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
        else:
            break

    return best_val_accuracy, classifier, lr, hidden_units, momentum, batch_size, weight_decay, best_model_state, model, criterion

In [ ]:
def test_model(model, model_state, criterion):
    model.load_state_dict(model_state)
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(test_loader, desc="Testing"):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            with amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_loss = running_loss / len(test_loader)
    test_accuracy = correct / total
    return test_loss, test_accuracy

In [ ]:
# Grid search sur tous les hyperparamètres et classificateurs
best_overall_accuracy = 0.0
best_overall_model = None
best_params = None

# Initialiser le fichier CSV et écrire les en-têtes
results_file = os.path.join(results_path, 'grid_search_results.csv')
with open(results_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Classifier', 'Learning Rate', 'Hidden Units', 'Momentum', 'Batch Size', 'Weight Decay', 'Validation Accuracy', 'Test Accuracy'])

for classifier, lr, hidden_units, momentum, batch_size, weight_decay in itertools.product(classifiers, learning_rates, hidden_units_list, momentums, batch_sizes, weight_decays):
    val_accuracy, classifier, lr, hidden_units, momentum, batch_size, weight_decay, model_state, model, criterion = train_model(classifier, lr, hidden_units, momentum, batch_size, weight_decay)

    if val_accuracy > best_overall_accuracy:
        test_loss, test_accuracy = test_model(model, model_state, criterion)
        print(f'Test Loss: {test_loss:.3f} Accuracy: {test_accuracy:.3f}')

        if test_accuracy > best_overall_accuracy:
            best_overall_accuracy = test_accuracy
            best_overall_model = model_state
            best_params = (classifier, lr, hidden_units, momentum, batch_size, weight_decay)

            # Sauvegarder le meilleur modèle
            best_model_save_path = os.path.join(models_path, f"best_model_{classifier}_lr_{best_params[1]}_hu_{best_params[2]}_momentum_{best_params[3]}_batch_{best_params[4]}_wd_{best_params[5]}_acc_{best_overall_accuracy:.3f}.pth")
            torch.save(best_overall_model, best_model_save_path)
            print(f'Best model saved as {best_model_save_path}')
            print(f'Best parameters: learning_rate={best_params[1]}, hidden_units={best_params[2]}, momentum={best_params[3]}, batch_size={best_params[4]}, weight_decay={best_params[5]}')
            print(f'Best validation accuracy: {best_overall_accuracy:.3f}')

    # Écrire les résultats de cette itération dans le fichier CSV
    with open(results_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([classifier, lr, hidden_units, momentum, batch_size, weight_decay, val_accuracy, test_accuracy])

print(f'Best Test Accuracy: {best_overall_accuracy:.3f}')
print(f'Best Parameters: Classifier={best_params[0]}, Learning Rate={best_params[1]}, Hidden Units={best_params[2]}, Momentum={best_params[3]}, Batch Size={best_params[4]}, Weight Decay={best_params[5]}')

Training with classifier=EfficientNet_V2_S_Weights, learning_rate=0.001, hidden_units=256, momentum=0.95, batch_size=16, weight_decay=0.0001


Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:01<00:00, 66.4MB/s]
Training Epoch 1: 100%|██████████| 86/86 [00:10<00:00,  8.08it/s]


Epoch 1 Train Loss: 0.730 Accuracy: 0.730


Validation: 100%|██████████| 22/22 [00:03<00:00,  6.70it/s]


Validation Loss: nan Accuracy: 0.880


...

Epoch 4 Train Loss: 0.234 Accuracy: 0.911


Validation: 100%|██████████| 6/6 [00:03<00:00,  1.74it/s]


Validation Loss: 0.209 Accuracy: 0.915


Testing: 100%|██████████| 10/10 [00:02<00:00,  4.97it/s]

Test Loss: 0.526 Accuracy: 0.830
Best Test Accuracy: 0.860
Best Parameters: Classifier=EfficientNet_V2_S_Weights, Learning Rate=0.01, Hidden Units=512, Momentum=0.95, Batch Size=32, Weight Decay=0.0001
